# Analysis of dataset to assign label

The purpose of this subproject is ; in a simple and easy way; assign a better label for classification in kind of disaster and whether or not it is a joke.

As dataset is pre-labeled from human team; i will use this features like basis and guide for this process.

## Preparing enviroment to work

In [1]:
!pip install boto3
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from boto3 import Session

In [3]:
disaster_db_df = pd.read_csv("/database.csv", encoding="latin9")
disaster_db_df.head(5)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one,choose_one:confidence,choose_one_gold,keyword,location,text,tweetid,userid
0,778243823,True,golden,156,NaN,Relevant,1.0000,Relevant,NaN,NaN,Just happened a terrible car crash,1.0,NaN
1,778243824,True,golden,152,NaN,Relevant,1.0000,Relevant,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,13.0,NaN
2,778243825,True,golden,137,NaN,Relevant,1.0000,Relevant,NaN,NaN,"Heard about #earthquake is different cities, s...",14.0,NaN
3,778243826,True,golden,136,NaN,Relevant,0.9603,Relevant,NaN,NaN,"there is a forest fire at spot pond, geese are...",15.0,NaN
4,778243827,True,golden,138,NaN,Relevant,1.0000,Relevant,NaN,NaN,Forest fire near La Ronge Sask. Canada,16.0,NaN


Based in data sample, i think the `choose_one`, `choose_one:confidence`, `choosen_one_gold`, `text` and `tweet_id` are important for the analysis; i need to check if my hypothesis is true.

---



# Checking the mentioned features are relevant or not

### `Choose_one` field

In [11]:
disaster_db_df['choose_one'].value_counts()

Not Relevant    6187
Relevant        4673
Can't Decide      16
Name: choose_one, dtype: int64

In [16]:
disaster_db_df['choose_one'].isnull().value_counts()

False    10876
Name: choose_one, dtype: int64

That's ready to use ✅

### `choose_one_gold` field

In [12]:
disaster_db_df['choose_one_gold'].value_counts()

Relevant                      57
Not Relevant                  24
Relevant\nCan't Decide         4
Not Relevant\nCan't Decide     2
Name: choose_one_gold, dtype: int64

In [15]:
disaster_db_df['choose_one_gold'].isnull().value_counts()

True     10789
False       87
Name: choose_one_gold, dtype: int64

***QUESTION #1: Can i replace it with `choose_one` field?***

In [33]:
disaster_db_df.replace(np.nan, None)
disaster_db_df["choose_one_gold_v2"] = disaster_db_df["choose_one_gold"].apply(
    lambda x: x.split("\n")[-1] if type(x) is str else None
)
disaster_db_df["choose_one_gold_v2"].value_counts()

Relevant        57
Not Relevant    24
Can't Decide     6
Name: choose_one_gold_v2, dtype: int64

In [47]:
disaster_db_df["is_choose_one_gold_with_choose_one"] = disaster_db_df.apply( 
    lambda x: True if x["choose_one_gold_v2"] == x["choose_one_gold"] else False, 
    axis=1
  )
selected_choose_one_gold_df = disaster_db_df[ disaster_db_df["choose_one_gold_v2"].isin(["Relevant", "Not Relevant", "Can't Decide"])]
selected_choose_one_gold_df["is_choose_one_gold_with_choose_one"].value_counts()

True     81
False     6
Name: is_choose_one_gold_with_choose_one, dtype: int64

In [48]:
selected_choose_one_gold_df = selected_choose_one_gold_df[selected_choose_one_gold_df["is_choose_one_gold_with_choose_one"].isin([False])]
selected_choose_one_gold_df[["choose_one_gold_v2", "choose_one", "choose_one_gold", "is_choose_one_gold_with_choose_one"]]

,choose_one_gold_v2,choose_one,choose_one_gold,is_choose_one_gold_with_choose_one
9,Can't Decide,Relevant,Relevant\nCan't Decide,False
13,Can't Decide,Relevant,Relevant\nCan't Decide,False
15,Can't Decide,Relevant,Relevant\nCan't Decide,False
19,Can't Decide,Relevant,Relevant\nCan't Decide,False
42,Can't Decide,Not Relevant,Not Relevant\nCan't Decide,False
44,Can't Decide,Not Relevant,Not Relevant\nCan't Decide,False


In previous table, i see that all non-null value of `choose_one_gold` field are same with `choose_one` field.

Ok, i ***won't use this field*** because it have several null value and i have more value from `choose_one` ❌

### `choose_one:confidence` field